In [1]:
import pandas as pd
import numpy as np
import scipy as sc
import re

In [2]:
TACmonoParameters = pd.read_csv("_data/TACmonoparameters.csv", sep=";")

In [3]:
TACmonoParameters.columns.values

array(['subject_nr', 'Leefpat', 'Age Donor', 'Type Donor',
       'Sex Recipient', 'Sex Donor', 'Total transplants',
       'Primary Diagnosis', 'PRA High Recipient',
       'Last dialysis technique', 'Cold Ischemia Time Hours',
       'Cold Ischemia Time Minutes',
       'Second Warm Ischemia Period in minutes', 'mm-A', 'mm-B', 'mm-DR',
       'mmtotaal', 'stopreden', 'REJECTIONWHENweeks', 'REJECTION_THERAPY',
       'TYPE_ANTIREJ', 'ENDP_REJECTION_TYPE', 'NOTR Date rej Treatment',
       'NOTR rej Treatment', 'aantal Aes', 'infwithAB', 'm6protbx',
       'TACdosem6', 'tacLEVELm6', 'MMFdosem6', 'MMFLEVELm6',
       'M6creatTACmonodb', 'GFRM6tacmonodb', 'M6urinepcrtacmonodb',
       'M6CKD_EPItacmonodb', 'NA_EXCRETIONm6', 'PROT_EXCRETIONm6',
       'K_EXCRETIONm6', 'M15creattacmono', 'M15GFRtacmono',
       'M15prctacmono', 'M15CKD_EPItacmono', 'M15NA_EXCRETION',
       'M15PROT_EXCRETION', 'M15FK_LEVEL', 'M15MMF_LEVEL',
       'M15K_EXCRETION', 'M3CKDnotr', 'M3creatnotr', 'M3protnotr'

In [4]:
targetList = ['rej'] #, 'nierf', 'nierfalen']
TACmonoParameters = TACmonoParameters[(TACmonoParameters.stopreden!='nierf') & (TACmonoParameters.stopreden!='nierfalen')]

In [5]:
TACmonoParameters['target'] = TACmonoParameters.stopreden.apply(lambda x: 1 if x in targetList else 0)

In [6]:
TACmonoParameters = TACmonoParameters[TACmonoParameters.stopreden!='anders']
TACmonoParameters = TACmonoParameters[TACmonoParameters.stopreden!='dood']
TACmonoParameters= TACmonoParameters[~TACmonoParameters.subject_nr.isna()]
TACmonoParameters.reset_index(inplace=True)

In [7]:
TACmonoAnalyse = pd.ExcelFile("_data/TACmonoAnalyse.xlsx")

In [8]:
TACmonoAnalyse_TC45 = pd.read_excel(TACmonoAnalyse, 'tabel TC45')
TACmonoAnalyse_TC45.dropna(axis=1,inplace=True,how='all')
TACmonoAnalyse_TC45.dropna(axis=0,inplace=True,how='all')

TACmonoAnalyse_BASIC = pd.read_excel(TACmonoAnalyse, 'tabel BASIC')
TACmonoAnalyse_BASIC.dropna(axis=1,inplace=True,how='all')
TACmonoAnalyse_BASIC.dropna(axis=0,inplace=True,how='all')

TACmonoAnalyse_Tcel= pd.read_excel(TACmonoAnalyse, 'tabel Tcel')
TACmonoAnalyse_Tcel.dropna(axis=1,inplace=True,how='all')
TACmonoAnalyse_Tcel.dropna(axis=0,inplace=True,how='all')


TACmonoAnalyse_Bcel= pd.read_excel(TACmonoAnalyse, 'tabel Bcel')
TACmonoAnalyse_Bcel.dropna(axis=1,inplace=True,how='all')
TACmonoAnalyse_Bcel.dropna(axis=0,inplace=True,how='all')


TACmonoAnalyse_DC= pd.read_excel(TACmonoAnalyse, 'tabel DC')
TACmonoAnalyse_DC.dropna(axis=1,inplace=True,how='all')
TACmonoAnalyse_DC.dropna(axis=0,inplace=True,how='all')

In [9]:
remove_trailing_number = re.compile(r'\.[0-9]$')
def newColName(x, colname):
    if 'Unnamed' not in colname:                
        # replace spaces by _
        if 'float' in str(type(x)):
            x=""   
        colname = remove_trailing_number.sub('', colname)
        out = colname+"_"+x
        return re.sub(r'\s', '_', out)
    else:
        if 'float' in str(type(x)):
            x=""            
        out = "_"+x
        return "empty" if x=="" else re.sub(r'\s', '_', out)  

In [10]:
for colname in TACmonoAnalyse_TC45.columns.values:
    TACmonoAnalyse_TC45.at[0, colname] = newColName(TACmonoAnalyse_TC45.at[0, colname], colname)
    
for colname in TACmonoAnalyse_BASIC.columns.values:
    TACmonoAnalyse_BASIC.at[0, colname] = newColName(TACmonoAnalyse_BASIC.at[0, colname], colname)
    
for colname in TACmonoAnalyse_Tcel.columns.values:
    TACmonoAnalyse_Tcel.at[0, colname] = newColName(TACmonoAnalyse_Tcel.at[0, colname], colname)

for colname in TACmonoAnalyse_Bcel.columns.values:
    TACmonoAnalyse_Bcel.at[0, colname] = newColName(TACmonoAnalyse_Bcel.at[0, colname], colname)
    
for colname in TACmonoAnalyse_DC.columns.values:
    TACmonoAnalyse_DC.at[0, colname] = newColName(TACmonoAnalyse_DC.at[0, colname], colname)

In [11]:
TACmonoAnalyse_TC45.columns = TACmonoAnalyse_TC45.iloc[0, :].values
TACmonoAnalyse_TC45.drop(0, axis=0, inplace=True)

TACmonoAnalyse_BASIC.columns = TACmonoAnalyse_BASIC.iloc[0, :].values
TACmonoAnalyse_BASIC.drop(0, axis=0, inplace=True)

TACmonoAnalyse_Tcel.columns = TACmonoAnalyse_Tcel.iloc[0, :].values
TACmonoAnalyse_Tcel.drop(0, axis=0, inplace=True)

TACmonoAnalyse_Bcel.columns = TACmonoAnalyse_Bcel.iloc[0, :].values
TACmonoAnalyse_Bcel.drop(0, axis=0, inplace=True)

TACmonoAnalyse_DC.columns = TACmonoAnalyse_DC.iloc[0, :].values
TACmonoAnalyse_DC.drop(0, axis=0, inplace=True)

In [12]:
TACmonoAnalyse_TC45 = TACmonoAnalyse_TC45[TACmonoAnalyse_TC45.studie_fase_=='in']
TACmonoAnalyse_TC45.drop('studie_fase_', axis=1, inplace=True)
TACmonoAnalyse_BASIC = TACmonoAnalyse_BASIC[TACmonoAnalyse_BASIC.studie_fase_=='in']
TACmonoAnalyse_BASIC.drop('studie_fase_', axis=1, inplace=True)
TACmonoAnalyse_Tcel = TACmonoAnalyse_Tcel[TACmonoAnalyse_Tcel.studie_fase_=='in']
TACmonoAnalyse_Tcel.drop('studie_fase_', axis=1, inplace=True)
TACmonoAnalyse_Bcel = TACmonoAnalyse_Bcel[TACmonoAnalyse_Bcel.studie_fase_=='in']
TACmonoAnalyse_Bcel.drop('studie_fase_', axis=1, inplace=True)
TACmonoAnalyse_DC = TACmonoAnalyse_DC[TACmonoAnalyse_DC.studie_fase_=='in']
TACmonoAnalyse_DC.drop('studie_fase_', axis=1, inplace=True)

In [13]:
def mapTime(x):
    if 'pre' in x:
        return 0
    elif 'M6' in x:
        return 1
    elif 'M9' in x: 
        return 2
    elif 'M12' in x:
        return 3
    elif 'M15' in x:
        return 4

TACmonoAnalyse_TC45.tijdspunt_ = TACmonoAnalyse_TC45.tijdspunt_.apply(lambda x: mapTime(x))
TACmonoAnalyse_BASIC.tijdspunt_ = TACmonoAnalyse_BASIC.tijdspunt_.apply(lambda x: mapTime(x))
TACmonoAnalyse_Tcel.tijdspunt_ = TACmonoAnalyse_Tcel.tijdspunt_.apply(lambda x: mapTime(x))
TACmonoAnalyse_Bcel.tijdspunt_ = TACmonoAnalyse_Bcel.tijdspunt_.apply(lambda x: mapTime(x))
TACmonoAnalyse_DC.tijdspunt_ = TACmonoAnalyse_DC.tijdspunt_.apply(lambda x: mapTime(x))

In [14]:
TACmonoAnalyse_TC45= TACmonoAnalyse_TC45[~TACmonoAnalyse_TC45.subject_nr_.isna()]

TACmonoAnalyse_BASIC=TACmonoAnalyse_BASIC[~TACmonoAnalyse_BASIC.subject_nr_.isna()]

TACmonoAnalyse_Tcel= TACmonoAnalyse_Tcel[~TACmonoAnalyse_Tcel.subject_nr_.isna()]

TACmonoAnalyse_Bcel= TACmonoAnalyse_Bcel[~TACmonoAnalyse_Bcel.subject_nr_.isna()]

TACmonoAnalyse_DC= TACmonoAnalyse_DC[~TACmonoAnalyse_DC.subject_nr_.isna()]

In [15]:
TACmonoAnalyse_TC45.columns = ['TC45_'+col for col in TACmonoAnalyse_TC45.columns.values]

TACmonoAnalyse_BASIC.columns = ['BASIC_'+col for col in TACmonoAnalyse_BASIC.columns.values]

TACmonoAnalyse_Tcel.columns = ['Tcel_'+col for col in TACmonoAnalyse_Tcel.columns.values]

TACmonoAnalyse_Bcel.columns = ['Bcel_'+col for col in TACmonoAnalyse_Bcel.columns.values]

TACmonoAnalyse_DC.columns = ['DC_'+col for col in TACmonoAnalyse_DC.columns.values]


In [16]:
TACmonoParameters.index = TACmonoParameters.subject_nr
TACmonoAnalyse_TC45.index = TACmonoAnalyse_TC45.TC45_subject_nr_
TACmonoAnalyse_BASIC.index = TACmonoAnalyse_BASIC.BASIC_subject_nr_
TACmonoAnalyse_Tcel.index = TACmonoAnalyse_Tcel.Tcel_subject_nr_
TACmonoAnalyse_Bcel.index = TACmonoAnalyse_Bcel.Bcel_subject_nr_
TACmonoAnalyse_DC.index = TACmonoAnalyse_DC.DC_subject_nr_


## Merge...

In [17]:
Final = pd.concat([TACmonoParameters, 
                  TACmonoAnalyse_TC45[TACmonoAnalyse_TC45.TC45_tijdspunt_==0],
                  TACmonoAnalyse_BASIC[TACmonoAnalyse_BASIC.BASIC_tijdspunt_==0],
                  TACmonoAnalyse_Tcel[TACmonoAnalyse_Tcel.Tcel_tijdspunt_==0],
                  TACmonoAnalyse_Bcel[TACmonoAnalyse_Bcel.Bcel_tijdspunt_==0],
                  TACmonoAnalyse_DC[TACmonoAnalyse_DC.DC_tijdspunt_==0]],
                  join='outer', axis=1)
Final.dropna(axis=0,inplace=True,how='all')
Final = Final[~Final.subject_nr.isna()]

In [18]:
Final.drop('index', axis=1, inplace=True)

In [19]:
for col in Final.columns.values:
    if "number" in col.lower():
        Final = Final.drop(col,axis=1)

In [20]:
for col in Final.columns.values:
    if "subject_nr" in col:
        Final = Final.drop(col,axis=1)

In [21]:
for col in Final.columns.values:
    if "studienr" in col:
        Final = Final.drop(col,axis=1)

In [22]:
for col in Final.columns.values:
    if "comment" in col.lower():
        Final = Final.drop(col,axis=1)

In [23]:
for col in Final.columns.values:
    if "complicaties" in col.lower():
        Final = Final.drop(col,axis=1)

In [24]:
for col in Final.columns.values:
    if "date" in col.lower():
        Final = Final.drop(col,axis=1)

In [25]:
for col in Final.columns.values:
    if "rejection" in col.lower():
        Final = Final.drop(col,axis=1)

In [26]:
for col in Final.columns.values:
    if "stopreden" in col.lower():
        Final = Final.drop(col,axis=1)

In [27]:
for col in Final.columns.values:
    if "Y1" in col:
        Final = Final.drop(col,axis=1)
for col in Final.columns.values:
    if "Y2" in col:
        Final = Final.drop(col,axis=1)
for col in Final.columns.values:
    if "M3" in col:
        Final = Final.drop(col,axis=1)
for col in Final.columns.values:
    if "M6" in col:
        Final = Final.drop(col,axis=1)        
for col in Final.columns.values:
    if "M15" in col:
        Final = Final.drop(col,axis=1)        

In [28]:
Final.target = Final.target.astype('int')

In [29]:
for col in Final.columns.values:
    try:
        if 'object' in str(Final[col].dtypes):
            try:
                Final[col] = Final[col].astype('float64')                
            except:
                try:
                    Final[col] = Final[col].astype('category')
                except:
                    Final[col] = Final[col].astype('str')
    except:
        Final[col] = Final[col].astype('category')

In [30]:
#Final['studienr']=Final.index

In [31]:
Final.to_csv("Prep_TACmono.csv",sep=";")

In [92]:
Final.columns.values

array(['Leefpat', 'Age Donor', 'Type Donor', 'Sex Recipient', 'Sex Donor',
       'Total transplants', 'Primary Diagnosis', 'PRA High Recipient',
       'Last dialysis technique', 'Cold Ischemia Time Hours',
       'Cold Ischemia Time Minutes',
       'Second Warm Ischemia Period in minutes', 'mm-A', 'mm-B', 'mm-DR',
       'mmtotaal', 'TYPE_ANTIREJ', 'NOTR rej Treatment', 'aantal Aes',
       'infwithAB', 'm6protbx', 'TACdosem6', 'tacLEVELm6', 'MMFdosem6',
       'MMFLEVELm6', 'NA_EXCRETIONm6', 'PROT_EXCRETIONm6',
       'K_EXCRETIONm6', 'NOTR m3.Immunosuppressive Drug 1',
       'NOTR m3.Immunosuppressive Drug 2',
       'NOTR m3.Immunosuppressive Drug 3',
       'NOTR y1.Immunosuppressive Drug 1',
       'NOTR y1.Immunosuppressive Drug 2',
       'NOTR y1.Immunosuppressive Drug 3', 'target', 'TC45_tijdspunt_',
       'TC45_all_%Gated', 'TC45_beads_%Gated', 'TC45_CD45_%Gated',
       'TC45_lymfocytes_%Gated', 'TC45_CD45_absoluut',
       'TC45_lymfocytes__absoluut', 'TC45_bead_count_

# ...train model

In [407]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split, cross_val_score  
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

In [408]:
df = pd.get_dummies(Final)
df.fillna(0, inplace=True)

In [462]:
x = df.drop(['target'], axis=1)
y = df['target']

In [463]:
for col in x.columns.values:
    if "tijdspunt" in col.lower():
        x = x.drop(col,axis=1)

In [464]:
# alleen klinische data
x=x[['Leefpat', 'Age Donor', 'Total transplants', 'PRA High Recipient',
       'Cold Ischemia Time Hours', 'Cold Ischemia Time Minutes',
       'Second Warm Ischemia Period in minutes', 'mm-A', 'mm-B', 'mm-DR',
       'mmtotaal', 'aantal Aes', 'infwithAB']]

In [ ]:
# PCA
x = PCA(n_components=50).fit_transform(x)

In [ ]:
# recursive feature elimination
MODEL = ensemble.RandomForestClassifier()
x = RFECV(step=1, cv=7, n_jobs=8, estimator=MODEL).fit_transform(x,y)

In [467]:
MODEL = ensemble.RandomForestClassifier(n_estimators=20, 
                                        min_samples_split=10, 
                                        max_depth=14, 
                                        min_samples_leaf=2,
                                        n_jobs=8, 
                                        bootstrap=True,
                                        criterion='entropy')
cv_results = cross_val_predict(MODEL, x, y, cv = 10)

print(metrics.classification_report(y, cv_results))
print("Total accuracy: {}".format(metrics.accuracy_score(y, cv_results)))
print("AUC/ROC: {}".format(metrics.roc_auc_score(y, cv_results)))

             precision    recall  f1-score   support

        0.0       0.84      0.97      0.90        78
        1.0       0.50      0.12      0.20        16

avg / total       0.79      0.83      0.78        94

Total accuracy: 0.8297872340425532
AUC/ROC: 0.5496794871794871


In [466]:
MODEL = ensemble.RandomForestClassifier(bootstrap=True, 
                                        criterion='gini', 
                                        max_features=0.9, 
                                        min_samples_leaf=12, 
                                        min_samples_split=9, 
                                        n_estimators=20)
cv_results = cross_val_predict(MODEL, x, y, cv = 10)

print(metrics.classification_report(y, cv_results))
print("Total accuracy: {}".format(metrics.accuracy_score(y, cv_results)))
print("AUC/ROC: {}".format(metrics.roc_auc_score(y, cv_results)))

             precision    recall  f1-score   support

        0.0       0.83      1.00      0.91        78
        1.0       0.00      0.00      0.00        16

avg / total       0.69      0.83      0.75        94

Total accuracy: 0.8297872340425532
AUC/ROC: 0.5


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# ExtraTreesClassifier(input_matrix, bootstrap=False, criterion=gini, max_features=0.6000000000000001, min_samples_leaf=3, min_samples_split=8, n_estimators=100)
# BernoulliNB(PCA(BernoulliNB(input_matrix, alpha=10.0, fit_prior=True), iterated_power=5, svd_solver=randomized), alpha=10.0, fit_prior=True)
# KNeighborsClassifier(input_matrix, n_neighbors=9, p=1, weights=distance)
# ensemble.RandomForestClassifier(bootstrap=False, criterion='entropy', max_features=0.7000000000000001, min_samples_leaf=14, min_samples_split=19, n_estimators=100)
# ensemble.RandomForestClassifier(bootstrap=True, criterion=gini, max_features=0.5, min_samples_leaf=10, min_samples_split=19, n_estimators=20)
# ensemble.RandomForestClassifier(bootstrap=True, criterion=gini, max_features=0.9000000000000001, min_samples_leaf=12, min_samples_split=9, n_estimators=20)

In [347]:
#from sklearn import pipeline
#pipeline.make_pipeline()

Pipeline(memory=None,
     steps=[('randomforestclassifier', RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None,
            max_features=0.7000000000000001, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=14, min_samples_split=19,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [430]:
acc = np.zeros(shape=y.shape)
count=0
maxrun = 50
run = True
importances_sum = 0 
tpot_config = {'sklearn.ensemble.RandomForestClassifier': {
                'n_estimators': [20,40],
                'criterion': ["gini", "entropy"],
                'max_features': np.arange(0.05, 1.01, 0.05),
                'min_samples_split': range(2, 21),
                'min_samples_leaf':  range(1, 21),
                'bootstrap': [True, False]}}
tpot = TPOTClassifier(generations=4, population_size=10, verbosity=2, config_dict=tpot_config)
while run:
    splitter = StratifiedKFold(10)
    #
    for train_index, test_index in splitter.split(x, y):
        x_train, x_test = np.array(x)[train_index], np.array(x)[test_index]
        y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
        tpot.fit(x_train, y_train)
        pred_test = tpot.predict(x_test)
        acc[test_index] = tpot.score(y_test, pred_test)    

        importances = MODEL.feature_importances_
        importances_sum += importances
        
    count+=1        
    if count>maxrun:
        run=False

print("Mean acc:{}, Var acc:{}".format(np.mean(acc), np.var(acc)))
indices = np.argsort(importances_sum)[::-1]    

for f in range(x.shape[1]):
    print("%d. %s (%f)" % (f + 1, x.columns[indices[f]], importances_sum[indices[f]]))

Optimization Progress:  40%|████      | 20/50 [00:11<00:16,  1.79pipeline/s]

Generation 1 - Current best internal CV score: 0.8786764705882353


Optimization Progress:  60%|██████    | 30/50 [00:16<00:10,  1.85pipeline/s]

Generation 2 - Current best internal CV score: 0.8786764705882353


Optimization Progress:  80%|████████  | 40/50 [00:21<00:04,  2.08pipeline/s]

Generation 3 - Current best internal CV score: 0.8786764705882353


Generation 4 - Current best internal CV score: 0.8904411764705882

Best pipeline: RandomForestClassifier(CombineDFs(input_matrix, input_matrix), bootstrap=True, criterion=gini, max_features=0.9000000000000001, min_samples_leaf=12, min_samples_split=9, n_estimators=20)


ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. 0. 0. 0. 0. 0. 1. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [468]:
acc = np.zeros(shape=y.shape)
count=0
maxrun = 50
run = True
importances_sum = 0 

while run:
    splitter = StratifiedKFold(10)
    #
    for train_index, test_index in splitter.split(x, y):
        x_train, x_test = np.array(x)[train_index], np.array(x)[test_index]
        y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]       
        MODEL.fit(x_train,y_train)
        pred_test_ = MODEL.predict(x_test) #[np.round(l[1]).astype(int) for l in pred_test]
        acc[test_index] = metrics.accuracy_score(y_test, pred_test_)    

        importances = MODEL.feature_importances_
        importances_sum += importances
        
    count+=1        
    if count>maxrun:
        run=False

print("Mean acc:{}, Var acc:{}".format(np.mean(acc), np.var(acc)))
indices = np.argsort(importances_sum)[::-1]    

for f in range(x.shape[1]):
    print("%d. %s (%f)" % (f + 1, x.columns[indices[f]], importances_sum[indices[f]]))

Mean acc:0.8297872340425532, Var acc:0.004154091846486595
1. aantal Aes (122.001862)
2. PRA High Recipient (66.753781)
3. Leefpat (53.346514)
4. Second Warm Ischemia Period in minutes (51.587360)
5. Cold Ischemia Time Minutes (50.427091)
6. Cold Ischemia Time Hours (45.040689)
7. Age Donor (43.329671)
8. mm-B (27.775590)
9. mmtotaal (19.279150)
10. mm-DR (14.216255)
11. infwithAB (8.346630)
12. mm-A (6.752972)
13. Total transplants (1.142434)


In [469]:
MODEL = LogisticRegressionCV()
cv_results = cross_val_predict(MODEL, x, y, cv = 10)
print(metrics.classification_report(y, cv_results))
print("Total accuracy: {}".format(metrics.accuracy_score(y, cv_results)))
print("AUC/ROC: {}".format(metrics.roc_auc_score(y, cv_results)))

             precision    recall  f1-score   support

        0.0       0.83      0.92      0.87        78
        1.0       0.14      0.06      0.09        16

avg / total       0.71      0.78      0.74        94

Total accuracy: 0.776595744680851
AUC/ROC: 0.49278846153846156


In [470]:
MODEL = LogisticRegressionCV()
MODEL.fit(x,y)
res = MODEL.coef_[0,:]

In [471]:
importances_df = pd.DataFrame()
importances_df['param'] = x.columns[indices[:]]
importances_df['importances'] = importances_sum[indices[:]]
importances_df.index = importances_df.param
importances_df.drop('param', axis=1, inplace=True)

In [472]:
result = pd.DataFrame()
result['coeff'] = 0 
result['coeff'] = res
result['param'] = ''
result['param'] = x.columns.values
result.index= result['param']
result.drop('param', axis=1, inplace=True)

In [474]:
result_final = importances_df.join(result, how='outer')
result_final = result_final.apply(lambda x: x/x.abs().max())
result_final = result_final.sort_values(by='importances', ascending=False)
result_final.to_csv('TACmono.csv')